# The GRAND WATERSHED PIPELINE

This notebook is designed to provide a parameterized pipeline for Watershed Transformations.  
Hopefully this wil simplify experimentation with different parameters, filtering functions,  
and input images.

In [1]:
from skimage import exposure
from skimage import filters
from skimage import io
import scipy.ndimage as ndi
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
"""
Parameters
"""
# on/off switch for inline images
SHOW_IMAGES = False

# path to input image
# using pre-cropped and pre-equalized band_contrast_image as input
ROOT_IMAGE_FILE = 'band_contrast_crop_equalize.png'

# on/off switch for histogram equalization of root_image
EQUALIZE_HIST = False

# rank order filter box size
RO_DIMENSIONS = 5

# local thresholding block size
LT_BLOCK_SIZE = 25


In [3]:
"""
Initial Image: root_image
Optional histogram equalization
"""
root_image = io.imread(ROOT_IMAGE_FILE, as_gray=True)

if EQUALIZE_HIST:
    root_image = exposure.equalize_hist(root_image)

print(f"root_image: {root_image.shape} {root_image.dtype}")

if SHOW_IMAGES:
    fig, axs = plt.subplots(1, figsize=(24, 12))
    axs.imshow(root_image, cmap='gray')


root_image: (3272, 1883) uint8


In [4]:
"""
Median Rank Filtering: rank_filtered
"""
median_rank = (RO_DIMENSIONS * RO_DIMENSIONS) // 2
rank_filtered = ndi.rank_filter(
    root_image, median_rank, size=(RO_DIMENSIONS, RO_DIMENSIONS)
)

print(f"rank_filtered: {rank_filtered.shape} {rank_filtered.dtype}")

if SHOW_IMAGES:
    fig, axs = plt.subplots(1, figsize=(24, 12))
    axs.imshow(rank_filtered, cmap='gray')


rank_filtered: (3272, 1883) uint8


In [5]:
"""
Local Thresholding of rank_filtered: local_thresh
Blobs detected between rank_filtered and local_thresh: blobs
"""
local_thresh = filters.threshold_local(rank_filtered, LT_BLOCK_SIZE)
blobs = rank_filtered > local_thresh

print(f"local_thresh: {local_thresh.shape} {local_thresh.dtype}")
print(f"blobs: {blobs.shape} {blobs.dtype}")

if SHOW_IMAGES:
    fig, axs = plt.subplots(1, figsize=(24, 12))
    axs.imshow(local_thresh, cmap='gray')
    fig, axs = plt.subplots(1, figsize=(24, 12))
    axs.imshow(blobs, cmap='gray')


local_thresh: (3272, 1883) float64
blobs: (3272, 1883) bool
